<a href="https://colab.research.google.com/github/jaeheonki/ICT-Capstone/blob/main/chatgpt%EC%A0%95%ED%99%95%EB%8F%84_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 38.1 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292701 sha256=d0d3d25b0937376a4b0a72177e4c3d3c8ceaa485d0b795968cf061a5137656ff
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, ht

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-zebs2a0a/kobert-tokenizer_411b57f6ed6b497a92526a4b5849b07f
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-zebs2a0a/kobert-tokenizer_411b57f6ed6b497a92526a4b5849b07f
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=da7c0026666e1c7815f605ed4392a176808ff08595b5954ae567df9cf42e772a
  Stored in directory: /tmp/pip-ephem-wheel-cache-7xkqat1u/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [ ]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
import inspect
import pandas as pd
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
chatbot_data = pd.read_csv('/content/drive/MyDrive/Chatgtp/최종모듈정리/per0 + gpt1 + feature final.csv')

In [ ]:
chatbot_data.loc[(chatbot_data['label'] == "0"), 'label'] = 0  #per => 0
chatbot_data.loc[(chatbot_data['label'] == "1"), 'label'] = 1  #gpt => 1


data_list = []
for q, label in zip(chatbot_data['text'], chatbot_data['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
# Setting parameters
max_len = 250
batch_size = 16
warmup_ratio = 0.1
num_epochs = 1
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
class newBERTSentenceTransform(nlp.data.BERTSentenceTransform):
  r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """
    
  def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab 

  def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)

        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = newBERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
from torch.optim import AdamW

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate) 
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
  
train_dataloader

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-24-480b6a139979>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/188 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7648890018463135 train acc 0.5625
epoch 1 train acc 0.930186170212766


<ipython-input-24-480b6a139979>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/63 [00:00<?, ?it/s]

epoch 1 test acc 0.9950396825396826


In [ ]:
#토큰화
#tokenizer = get_tokenizer()
#tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def sumsum(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tokenizer,vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                return 0
            elif np.argmax(logits) == 1:
                return 1

        

class 끝

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 

import missingno as msno 

from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Chatgtp/최종모듈정리/final.csv')

In [ ]:
data.shape

(4000, 11)

In [ ]:
data.columns

Index(['text', 'label', 'probability', 'classification', 'final_word_valplus',
       'final_word_valsub', 'final_sen_valplus', 'final_sen_valsub',
       'ad_num_word', 'ad_num_sen', 'var'],
      dtype='object')

In [ ]:
data.count()

text                  4000
label                 4000
probability           4000
classification        4000
final_word_valplus    4000
final_word_valsub     4000
final_sen_valplus     4000
final_sen_valsub      4000
ad_num_word           4000
ad_num_sen            4000
var                   4000
dtype: int64

In [ ]:
data["label"].value_counts()

0    2000
1    2000
Name: label, dtype: int64

In [ ]:
data[['probability', 'label']].groupby('label', as_index=True).mean()

,probability
label,
0,0.349451
1,0.374844


In [ ]:
data[['classification', 'label']].groupby('label', as_index=True).mean()

,classification
label,
0,0.0030
1,0.9965


In [ ]:
data[['final_word_valplus', 'label']].groupby('label', as_index=True).mean()

,final_word_valplus
label,
0,0.064104
1,0.027550


In [ ]:
data[['final_word_valsub','label']].groupby(['label'], as_index=True).mean()

,final_word_valsub
label,
0,0.042904
1,0.002625


In [ ]:
data[['final_sen_valplus', 'label']].groupby('label', as_index=True).mean()

,final_sen_valplus
label,
0,0.963769
1,0.617664


In [ ]:
data[['final_sen_valsub', 'label']].groupby('label', as_index=True).mean()

,final_sen_valsub
label,
0,0.836569
1,0.380364


In [ ]:
data[['ad_num_word', 'label']].groupby('label', as_index=True).mean()

,ad_num_word
label,
0,0.058954
1,0.040675


In [ ]:
data[['ad_num_sen', 'label']].groupby('label', as_index=True).mean()

,ad_num_sen
label,
0,0.932869
1,0.716064


In [ ]:
data[['var', 'label']].groupby('label', as_index=True).mean()

,var
label,
0,96.917149
1,36.171019


In [ ]:
data = data.drop(columns = ['text'])

In [ ]:
y = data.label.values
x_data = data.drop(['label'], axis = 1)

In [ ]:
x_data.shape

(4000, 9)

In [ ]:
y

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
x_data.columns

Index(['probability', 'classification', 'final_word_valplus',
       'final_word_valsub', 'final_sen_valplus', 'final_sen_valsub',
       'ad_num_word', 'ad_num_sen', 'var'],
      dtype='object')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y, test_size = 0.2, random_state = 0)

In [ ]:
x_train.shape

(3200, 9)

In [ ]:
y_train.shape

(3200,)

In [ ]:
x_test.shape

(800, 9)

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming you have X_train, X_test, y_train, y_test as your training and testing data

# Preprocess the data by standardizing it
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Create an SVM classifier
clf = svm.SVC(probability=True)

# Train the SVM model
clf.fit(x_train_scaled, y_train)

# Evaluate the model on the test set
accuracy = clf.score(x_test_scaled, y_test)
print("Accuracy: {:.2f}".format(accuracy))

Accuracy: 0.99


In [ ]:
text = input("텍스트를 입력하세요: ")
print('현재 텍스트:', text)

텍스트를 입력하세요: 자연재해는 인간의 생활에 큰 영향을 미칠 수 있는 불가피한 사건입니다. 홍수, 지진, 태풍 등의 자연재해는 인명 피해와 재산 피해를 초래할 수 있으며, 피해를 입은 지역의 복구에도 많은 시간과 노력이 필요합니다. 따라서, 자연재해에 대비하는 것은 매우 중요한 과제입니다. 정부와 지역사회는 이를 위해 예방조치와 대응체계를 강화하고, 주민들은 재해 대비를 위한 준비를 철저히 해야 합니다. 효과적인 대응과 복구를 위해서는 긴박한 상황에서도 조직적으로 협력하는 것이 필요하며, 국제적인 협력과 지원도 중요한 역할을 합니다. 자연재해는 예측하기 힘든 요소가 많지만, 우리는 이를 대비하는 노력을 지속적으로 기울여야 합니다.
현재 텍스트: 자연재해는 인간의 생활에 큰 영향을 미칠 수 있는 불가피한 사건입니다. 홍수, 지진, 태풍 등의 자연재해는 인명 피해와 재산 피해를 초래할 수 있으며, 피해를 입은 지역의 복구에도 많은 시간과 노력이 필요합니다. 따라서, 자연재해에 대비하는 것은 매우 중요한 과제입니다. 정부와 지역사회는 이를 위해 예방조치와 대응체계를 강화하고, 주민들은 재해 대비를 위한 준비를 철저히 해야 합니다. 효과적인 대응과 복구를 위해서는 긴박한 상황에서도 조직적으로 협력하는 것이 필요하며, 국제적인 협력과 지원도 중요한 역할을 합니다. 자연재해는 예측하기 힘든 요소가 많지만, 우리는 이를 대비하는 노력을 지속적으로 기울여야 합니다.


In [ ]:
classification = sumsum(text)

In [ ]:
!pip install nltk konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 41.1 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Hannanum

In [ ]:
# Import necessary modules
from collections import Counter
import nltk
from nltk.corpus import wordnet
import torch
import math

# Download the WordNet corpus if you haven't already
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from nltk.tokenize import sent_tokenize

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Instantiate the Hannanum class
hannanum = Hannanum()

In [ ]:
connecting_adverbs_list = ['아니면', '오직', '요컨데', '고로', '그러므로', '그런데', '그래서', '그리고', '혹은', '또는', '및', '그러나', '게다가', '더욱이', '더구나', '아울러', '뿐만 아니라', '동시에', '그런 점에서', '어쩌면', '하물며', '이처럼', '이같이', '바로', '하지만', '그렇지만', '그럼에도', '반면에', '도리어', '오히려', '반대로', '따라서', '그러니까', '그리하여', '그렇기 때문에', '그러면', '그러니', '급기야', '마침내', '왜냐하면', '다른 한편', '그렇기는 해도', '다만', '바꿔 말하면', '즉', '곧', '말하자면', '예를 들면', '일례로', '사실상', '예컨대', '덧붙여', '구체적으로', '왜냐하면', '이를테면', '다시 말하면', '끝으로', '결국', '결론적으로', '마지막으로', '요컨대', '결과적으로', '분명한 것은', '종합하면', '그런 다음']

In [ ]:
import os
import random
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

In [ ]:
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.8.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'lm_head.weight', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.7.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
sentences = sent_tokenize(text)
words = word_tokenize(text)
qualifying_sentences = []

pos_tags = hannanum.pos(text)

# Count connecting adverbs
connecting_adverb_counter = Counter()

# Set for storing unique connecting adverbs
unique_connecting_adverb_set = set()

# Check for connecting adverbs using their POS tags
for word, pos in pos_tags:
    if word in connecting_adverbs_list or pos == 'M':
        connecting_adverb_counter[word] += 1
        unique_connecting_adverb_set.add(word)

# Calculate the average number of unique adverbs per sentence
avg_adverbs_per_sentence = len(unique_connecting_adverb_set) / len(sentences)
avg_adverbs_per_word = len(unique_connecting_adverb_set) / len(words)

total_squared_diff = 0
total_average = len(words) / len(sentences)

# Calculate adjusted averages
if len(unique_connecting_adverb_set) >= 5 and avg_adverbs_per_sentence >= 1.0:
    avg_adverbs_per_sentence += 0.3
    avg_adverbs_per_word += 0.05

# Calculate variance and standard deviation
for sentence in sentences:
    words_in_sentence = word_tokenize(sentence)
    total_squared_diff += (len(words_in_sentence) - total_average) ** 2

    if -3 <= len(words_in_sentence) - total_average <= 3 and avg_adverbs_per_sentence < 0.5:
      qualifying_sentences.append(sentence)
    
total_variance = total_squared_diff / len(sentences)
total_standard_deviation = math.sqrt(total_variance)

# Check if the standard deviation exceeds a threshold
if total_standard_deviation <= 5:
    avg_adverbs_per_sentence -= 0.3
    avg_adverbs_per_word -= 0.05

print(f"Adjusted average number of unique connecting adverbs per sentence: {avg_adverbs_per_sentence}")
print(f"Adjusted average number of unique connecting adverbs per word: {avg_adverbs_per_word}")
print(f"total_variance: {total_variance}")

for sentence in qualifying_sentences:
    print(sentence)

final_sen_valsub = avg_adverbs_per_sentence
final_word_valsub = avg_adverbs_per_word
var = total_variance

Adjusted average number of unique connecting adverbs per sentence: 0.033333333333333326
Adjusted average number of unique connecting adverbs per word: -0.02849462365591398
total_variance: 24.25
정부와 지역사회는 이를 위해 예방조치와 대응체계를 강화하고, 주민들은 재해 대비를 위한 준비를 철저히 해야 합니다.
효과적인 대응과 복구를 위해서는 긴박한 상황에서도 조직적으로 협력하는 것이 필요하며, 국제적인 협력과 지원도 중요한 역할을 합니다.
자연재해는 예측하기 힘든 요소가 많지만, 우리는 이를 대비하는 노력을 지속적으로 기울여야 합니다.


In [ ]:
sentences = sent_tokenize(text)
words = word_tokenize(text)

pos_tags = hannanum.pos(text)

# Count connecting adverbs
connecting_adverb_counter = Counter()

# Set for storing unique connecting adverbs
unique_connecting_adverb_set = set()

# Check for connecting adverbs using their POS tags
for word, pos in pos_tags:
    if word in connecting_adverbs_list or pos == 'M':
        connecting_adverb_counter[word] += 1
        unique_connecting_adverb_set.add(word)

# Calculate the average number of unique adverbs per sentence
avg_adverbs_per_sentence2 = len(unique_connecting_adverb_set) / len(sentences)
avg_adverbs_per_word2 = len(unique_connecting_adverb_set) / len(words)

print("Average number of unique connecting adverbs per sentence:", avg_adverbs_per_sentence2)
print("Average number of unique connecting adverbs per word:", avg_adverbs_per_word2)

ad_num_sen = avg_adverbs_per_sentence2
ad_num_word = avg_adverbs_per_word2

Average number of unique connecting adverbs per sentence: 0.3333333333333333
Average number of unique connecting adverbs per word: 0.021505376344086023


In [ ]:
sentences = sent_tokenize(text)
words = word_tokenize(text)

pos_tags = hannanum.pos(text)

# Count connecting adverbs
connecting_adverb_counter = Counter()

# Set for storing unique connecting adverbs
unique_connecting_adverb_set = set()

# Check for connecting adverbs using their POS tags
for word, pos in pos_tags:
    if word in connecting_adverbs_list or pos == 'M':
        connecting_adverb_counter[word] += 1
        unique_connecting_adverb_set.add(word)

# Calculate the average number of unique adverbs per sentence
avg_adverbs_per_sentence3 = len(unique_connecting_adverb_set) / len(sentences)
avg_adverbs_per_word3 = len(unique_connecting_adverb_set) / len(words)

total_squared_diff = 0
total_average = len(words) / len(sentences)

# Calculate adjusted averages
if len(unique_connecting_adverb_set) >= 5 and avg_adverbs_per_sentence3 >= 1.0:
    avg_adverbs_per_sentence3 += 0.3
    avg_adverbs_per_word3 += 0.05

# Calculate variance and standard deviation
for sentence in sentences:
    words_in_sentence = word_tokenize(sentence)
    total_squared_diff += (len(words_in_sentence) - total_average) ** 2

total_variance = total_squared_diff / len(sentences)
total_standard_deviation = math.sqrt(total_variance)

# Check if the standard deviation exceeds a threshold
if total_standard_deviation >= 9:
    avg_adverbs_per_sentence3 += 0.3
    avg_adverbs_per_word3 += 0.05

print(f"Adjusted average number of unique connecting adverbs per sentence: {avg_adverbs_per_sentence3}")
print(f"Adjusted average number of unique connecting adverbs per word: {avg_adverbs_per_word3}")
print(f"total_variance: {total_variance}")

final_sen_valplus = avg_adverbs_per_sentence3
final_word_valplus = avg_adverbs_per_word3

Adjusted average number of unique connecting adverbs per sentence: 0.3333333333333333
Adjusted average number of unique connecting adverbs per word: 0.021505376344086023
total_variance: 24.25


In [ ]:
def calculate_probability(sent_ap, next_word):
    input_text = sent_ap + " " + next_word
    input_ids = tokenizer.encode(input_text, return_tensors="tf")
    output = model(input_ids)
    logits = output.logits[0, -1, :]
    softmax_output = tf.nn.softmax(logits).numpy().tolist()
    next_word_id = input_ids[0, -1].numpy()
    return softmax_output[next_word_id]

def calculate_sum(texting):
    sent_split = texting.split(" ")

    total_sum = 0
    sent_ap = ""
    for word in sent_split:
        next_word_probability = calculate_probability(sent_ap, word)
        sent_ap += " " + word
        total_sum += next_word_probability

    return round(total_sum * 1000 / len(sent_split), 5)

file_sum = calculate_sum(text)
print("Sum value for the text:", file_sum)
probability = file_sum

Sum value for the text: 0.16772


In [ ]:
abc = [probability, classification, final_word_valplus, final_word_valsub, final_sen_valplus, final_sen_valsub, ad_num_word, ad_num_sen, var]

array = np.array(abc)
array_reshaped = np.reshape(array, (1, 9))

new_sample_scaled = scaler.transform(array_reshaped)

probabilities = clf.predict_proba(new_sample_scaled)
classlabel = clf.predict(new_sample_scaled)

class_0_prob = probabilities[0, 0]
class_1_prob = probabilities[0, 1]

print("Probability of class 0: {:.2f}".format(class_0_prob))
print("Probability of class 1: {:.2f}".format(class_1_prob))
print(classlabel)

Probability of class 0: 0.00
Probability of class 1: 1.00
[1]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
print(new_sample_scaled)

[[-0.05431107 -1.00564091  0.29463411 -0.3518059  -0.07951483 -0.28413384
   0.29834031 -0.15578387 -0.21415233]]
